In [ ]:
# ============================================================================
# SIMPLE ALTERNATIVE (FALLBACK)
# ============================================================================

def simple_transcript_rag(youtube_url: str):
    """Simple RAG with just transcript - guaranteed to work"""
    print(f"🎬 Simple transcript processing: {youtube_url}")

    # Get basic info
    video_id = extract_video_id(youtube_url)
    transcript = get_youtube_transcript(video_id)

    if not transcript:
        print("❌ No transcript available")
        return None

    # Create simple documents
    documents = []
    for i, entry in enumerate(transcript):
        doc = Document(
            text=entry['text'],
            metadata={
                "timestamp": entry['start'],
                "timestamp_formatted": f"{int(entry['start']//60):02d}:{int(entry['start']%60):02d}",
                "duration": entry.get('duration', 3),
                "segment": i
            }
        )
        documents.append(doc)

    # Create simple index
    index = VectorStoreIndex.from_documents(documents, embed_model=Settings.embed_model)
    query_engine = index.as_query_engine(llm=gemini_llm)

    print(f"✅ Simple RAG ready with {len(documents)} segments")

    return {
        "query_engine": query_engine,
        "video_id": video_id,
        "transcript_count": len(documents)
    }


In [ ]:
# ============================================================================
# UTILITY FUNCTIONS FOR COLAB SESSIONS
# ============================================================================

def clear_chromadb():
    """Clear ChromaDB collections for fresh start"""
    try:
        chroma_client = chromadb.EphemeralClient()
        collections = chroma_client.list_collections()
        for collection in collections:
            chroma_client.delete_collection(collection.name)
            print(f"🧹 Deleted collection: {collection.name}")
        print("✅ ChromaDB cleared!")
    except Exception as e:
        print(f"⚠️  ChromaDB clear failed: {e}")

def restart_session():
    """Clean restart - clear data and temp files"""
    clear_chromadb()

    # Clear work directory
    import shutil
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)
        os.makedirs(WORK_DIR, exist_ok=True)
        print("🧹 Cleared work directory")

    print("✅ Session restarted - ready for fresh processing!")

In [ ]:
# LlamaIndex + Gemini Video Processing Pipeline
# Adapted from: https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/multi_modal/gemini.ipynb

# ============================================================================
# SETUP & INSTALLATION
# ============================================================================

!pip install -q llama-index llama-index-multi-modal-llms-gemini llama-index-vector-stores-chroma llama-index-embeddings-huggingface
!pip install -q youtube-transcript-api yt-dlp opencv-python chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.4 MB/s eta 0:00:0

In [ ]:
import os
import json
import tempfile
from typing import List, Dict, Any, Optional
import cv2
import numpy as np
from pathlib import Path

# LlamaIndex imports
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.schema import ImageDocument, Document
from llama_index.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.query_engine import SimpleMultiModalQueryEngine
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Other imports
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp
from google.colab import drive, userdata
import chromadb
from datetime import datetime
import re

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Get Gemini API key
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# Initialize Gemini MultiModal LLM
gemini_llm = GeminiMultiModal(
    model_name="models/gemini-1.5-flash-latest",  # Using flash for better rate limits
    api_key=GEMINI_API_KEY,
    temperature=0.1,
    max_tokens=1024
)

# Setup local embedding model (no OpenAI needed!)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Configure LlamaIndex to use local embeddings globally
Settings.embed_model = embed_model
Settings.llm = gemini_llm

<ipython-input-9-19d7a57aedcf>:10: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  gemini_llm = GeminiMultiModal(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Setup directories
WORK_DIR = "/content/video_processing"
OUTPUT_DIR = "/content/drive/MyDrive/VideoAnalysis"
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✅ LlamaIndex + Gemini setup complete!")

✅ LlamaIndex + Gemini setup complete!


In [ ]:
# ============================================================================
# VIDEO PROCESSING UTILITIES
# ============================================================================

def extract_video_id(youtube_url: str) -> Optional[str]:
    """Extract video ID from YouTube URL"""
    patterns = [
        r'(?:youtube\.com/watch\?v=|youtu\.be/)([a-zA-Z0-9_-]{11})',
        r'youtube\.com/embed/([a-zA-Z0-9_-]{11})',
    ]

    for pattern in patterns:
        match = re.search(pattern, youtube_url)
        if match:
            return match.group(1)
    return None

In [ ]:
def get_youtube_transcript(video_id: str) -> Optional[List[Dict]]:
    """Get transcript with timestamps"""
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"❌ Error getting transcript: {e}")
        return None

In [ ]:
def download_video_for_frames(youtube_url: str, max_duration: int = 300) -> Optional[Dict]:
    """Download video optimized for frame extraction"""
    try:
        ydl_opts = {
            'format': 'best[height<=720]',  # Reasonable quality
            'outtmpl': f'{WORK_DIR}/%(title)s.%(ext)s',
            'external_downloader_args': ['-t', str(max_duration)],  # Limit duration
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print("🔄 Downloading video for frame extraction...")
            info = ydl.extract_info(youtube_url, download=True)
            video_path = ydl.prepare_filename(info)

            return {
                'video_path': video_path,
                'title': info.get('title', 'Unknown'),
                'duration': info.get('duration', 0),
                'description': info.get('description', ''),
                'uploader': info.get('uploader', 'Unknown')
            }
    except Exception as e:
        print(f"❌ Error downloading video: {e}")
        return None

In [ ]:
def extract_frames_for_analysis(video_path: str, num_frames: int = 8) -> List[str]:
    """Extract frames at regular intervals for multimodal analysis"""
    try:
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = total_frames / fps if fps > 0 else 0

        frame_paths = []
        frames_dir = f"{WORK_DIR}/frames"
        os.makedirs(frames_dir, exist_ok=True)

        # Extract frames at regular intervals
        for i in range(num_frames):
            frame_time = (duration / num_frames) * i
            timestamp_seconds = frame_time

            # Set video position
            cap.set(cv2.CAP_PROP_POS_MSEC, frame_time * 1000)
            ret, frame = cap.read()

            if ret:
                # Save frame
                frame_filename = f"frame_{i:03d}_{int(timestamp_seconds):04d}s.jpg"
                frame_path = os.path.join(frames_dir, frame_filename)
                cv2.imwrite(frame_path, frame)
                frame_paths.append(frame_path)

                print(f"✅ Extracted frame at {timestamp_seconds:.1f}s")

        cap.release()
        return frame_paths

    except Exception as e:
        print(f"❌ Error extracting frames: {e}")
        return []

In [ ]:
# ============================================================================
# LLAMAINDEX MULTIMODAL RAG SETUP
# ============================================================================

def create_transcript_documents(transcript: List[Dict], video_info: Dict) -> List[Document]:
    """Create LlamaIndex documents from transcript with rich metadata"""
    documents = []

    # Create 30-second chunks with enhanced context
    current_chunk = {"start_time": 0, "text": "", "end_time": 0}

    for entry in transcript:
        # Start new chunk if we've exceeded 30 seconds
        if entry['start'] - current_chunk['start_time'] > 30:
            if current_chunk['text'].strip():
                # Create document with rich metadata
                doc = Document(
                    text=current_chunk['text'].strip(),
                    metadata={
                        "source_type": "transcript",
                        "video_title": video_info['title'],
                        "start_time": current_chunk['start_time'],
                        "end_time": current_chunk['end_time'],
                        "timestamp_formatted": f"{int(current_chunk['start_time']//60):02d}:{int(current_chunk['start_time']%60):02d}",
                        "duration_seconds": current_chunk['end_time'] - current_chunk['start_time'],
                        "video_duration": video_info['duration'],
                        "uploader": video_info.get('uploader', 'Unknown')
                    }
                )
                documents.append(doc)

            # Start new chunk
            current_chunk = {
                "start_time": entry['start'],
                "text": "",
                "end_time": entry['start']
            }

        # Add text to current chunk
        current_chunk['text'] += " " + entry['text'].strip()
        current_chunk['end_time'] = entry['start'] + entry.get('duration', 3)

    # Add final chunk
    if current_chunk['text'].strip():
        doc = Document(
            text=current_chunk['text'].strip(),
            metadata={
                "source_type": "transcript",
                "video_title": video_info['title'],
                "start_time": current_chunk['start_time'],
                "end_time": current_chunk['end_time'],
                "timestamp_formatted": f"{int(current_chunk['start_time']//60):02d}:{int(current_chunk['start_time']%60):02d}",
                "duration_seconds": current_chunk['end_time'] - current_chunk['start_time'],
                "video_duration": video_info['duration'],
                "uploader": video_info.get('uploader', 'Unknown')
            }
        )
        documents.append(doc)

    return documents

In [ ]:
def create_image_documents(frame_paths: List[str], video_info: Dict) -> List[ImageDocument]:
    """Create LlamaIndex image documents from extracted frames"""
    image_documents = []

    for i, frame_path in enumerate(frame_paths):
        # Extract timestamp from filename
        timestamp_match = re.search(r'(\d+)s\.jpg', frame_path)
        timestamp = int(timestamp_match.group(1)) if timestamp_match else i * 30

        # Create image document with metadata
        img_doc = ImageDocument(
            image_path=frame_path,
            metadata={
                "source_type": "video_frame",
                "video_title": video_info['title'],
                "frame_number": i,
                "timestamp": timestamp,
                "timestamp_formatted": f"{timestamp//60:02d}:{timestamp%60:02d}",
                "frame_path": frame_path,
                "video_duration": video_info['duration']
            }
        )
        image_documents.append(img_doc)

    return image_documents

In [ ]:
def setup_multimodal_rag(transcript_docs, image_docs):
    """Setup LlamaIndex multimodal RAG system"""

    # Setup ChromaDB for vector storage
    chroma_client = chromadb.EphemeralClient()

    # Use unique collection name or get existing one
    collection_name = f"video_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    try:
        # Try to delete existing collection if it exists
        try:
            chroma_client.delete_collection("video_analysis")
            print("🧹 Cleaned up existing collection")
        except:
            pass

        chroma_collection = chroma_client.create_collection(collection_name)
        print(f"✅ Created new collection: {collection_name}")

    except Exception as e:
        print(f"⚠️  Collection issue: {e}")
        # Fallback: get or create
        chroma_collection = chroma_client.get_or_create_collection(collection_name)
        print(f"✅ Using existing/new collection: {collection_name}")

    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Combine all documents for unified search
    all_documents = transcript_docs.copy()
    if image_docs:
        all_documents.extend(image_docs)

    # Create unified index with all content
    print("🔄 Creating unified multimodal index...")
    unified_index = VectorStoreIndex.from_documents(
        all_documents,
        storage_context=storage_context,
        embed_model=Settings.embed_model
    )

    # Create query engine with multimodal LLM
    query_engine = unified_index.as_query_engine(
        llm=gemini_llm,
        similarity_top_k=5,  # Get more results for better context
        response_mode="tree_summarize"  # Better for multimodal responses
    )

    print(f"✅ Created unified query engine with {len(all_documents)} documents")

    return query_engine, unified_index, None

In [ ]:
# ============================================================================
# MAIN PROCESSING PIPELINE
# ============================================================================

def process_youtube_video_llamaindex(youtube_url: str) -> Dict[str, Any]:
    """Process YouTube video using LlamaIndex + Gemini multimodal RAG"""

    print(f"\n🎬 Processing video with LlamaIndex + Gemini: {youtube_url}")
    print("=" * 70)

    # Step 1: Extract video ID
    video_id = extract_video_id(youtube_url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")

    # Step 2: Get transcript
    print("1️⃣ Getting transcript...")
    transcript = get_youtube_transcript(video_id)
    if not transcript:
        raise ValueError("Could not get transcript - video may not have captions")
    print(f"✅ Retrieved {len(transcript)} transcript segments")

    # Step 3: Download video and extract frames
    print("2️⃣ Downloading video and extracting frames...")
    video_info = download_video_for_frames(youtube_url)
    if not video_info:
        raise ValueError("Could not download video")

    frame_paths = extract_frames_for_analysis(video_info['video_path'])
    if not frame_paths:
        print("⚠️  No frames extracted, proceeding with transcript only")

    print(f"✅ Extracted {len(frame_paths)} frames")

    # Step 4: Create LlamaIndex documents
    print("3️⃣ Creating LlamaIndex documents...")
    transcript_docs = create_transcript_documents(transcript, video_info)
    image_docs = create_image_documents(frame_paths, video_info) if frame_paths else []

    print(f"✅ Created {len(transcript_docs)} transcript docs, {len(image_docs)} image docs")

    # Step 5: Setup multimodal RAG
    print("4️⃣ Setting up multimodal RAG system...")
    query_engine, text_index, image_index = setup_multimodal_rag(transcript_docs, image_docs)

    print("✅ Multimodal RAG system ready!")

    # Step 6: Save processing results
    result = {
        "video_id": video_id,
        "youtube_url": youtube_url,
        "video_info": video_info,
        "transcript": transcript,
        "frame_paths": frame_paths,
        "transcript_doc_count": len(transcript_docs),
        "image_doc_count": len(image_docs),
        "processing_metadata": {
            "processed_at": datetime.now().isoformat(),
            "method": "llamaindex_multimodal",
            "gemini_model": "gemini-1.5-flash-latest"
        }
    }

    # Save metadata (query engine is stored in memory for this session)
    output_file = f"{OUTPUT_DIR}/video_analysis_{video_id}.json"
    with open(output_file, 'w') as f:
        json.dump(result, f, indent=2)

    print(f"\n🎉 Processing complete!")
    print(f"💾 Metadata saved to: {output_file}")
    print(f"🧠 Query engine ready for search and chat!")

    # Clean up video file
    try:
        os.remove(video_info['video_path'])
    except:
        pass

    return {
        "metadata": result,
        "query_engine": query_engine,
        "text_index": text_index,
        "image_index": image_index
    }

In [ ]:
# ============================================================================
# USAGE FUNCTIONS
# ============================================================================

def test_multimodal_search(query_engine, query: str):
    """Test the multimodal search capability"""
    print(f"\n🔍 Testing search: '{query}'")
    print("-" * 50)

    try:
        response = query_engine.query(query)
        print(f"📝 Response: {response}")

        # Show source information
        if hasattr(response, 'source_nodes'):
            print(f"\n📚 Sources found: {len(response.source_nodes)}")
            for i, node in enumerate(response.source_nodes[:3]):  # Show top 3
                metadata = node.metadata
                if metadata.get('source_type') == 'transcript':
                    print(f"  {i+1}. Transcript at {metadata.get('timestamp_formatted', 'unknown')}")
                elif metadata.get('source_type') == 'video_frame':
                    print(f"  {i+1}. Frame at {metadata.get('timestamp_formatted', 'unknown')}")
                print(f"     Content: {node.text[:100]}...")

        return response
    except Exception as e:
        print(f"❌ Search error: {e}")
        return None


In [ ]:
def demo_video_analysis(youtube_url: str):
    """Complete demo of video analysis with sample queries"""

    # Process video
    result = process_youtube_video_llamaindex(youtube_url)
    query_engine = result["query_engine"]

    # Demo queries
    demo_queries = [
        "What is the main topic of this video?",
        "What objects can you see in the video?",
        "Summarize the key points discussed",
        "What colors are prominent in the video?",
        "When does the speaker mention specific technical terms?"
    ]

    print(f"\n🎯 Running demo queries...")
    print("=" * 50)

    for query in demo_queries[:3]:  # Test first 3 queries
        test_multimodal_search(query_engine, query)
        print()

    return result


In [ ]:
# ============================================================================
# READY TO USE
# ============================================================================

print("🚀 LlamaIndex + Gemini Video Processing Ready!")
print("=" * 50)
print("✅ Multimodal RAG with video frames + transcript")
print("✅ Using local embeddings (no OpenAI key needed)")
print("✅ Professional document management")
print("✅ Built-in error handling and retries")
print("✅ Sophisticated retrieval and ranking")
print()
print("Usage:")
print("result = demo_video_analysis('YOUR_YOUTUBE_URL')")
print()
print("💡 Perfect for AI startup demo!")

# Test that embeddings are working
print("\n🧪 Testing local embeddings...")
try:
    test_embedding = Settings.embed_model.get_text_embedding("This is a test.")
    print(f"✅ Local embeddings working! Vector dimension: {len(test_embedding)}")
except Exception as e:
    print(f"❌ Embedding test failed: {e}")

# Example usage:
restart_session()
result = demo_video_analysis("https://www.youtube.com/watch?v=kD3-DKkiVeA")
# result = simple_transcript_rag("https://www.youtube.com/watch?v=kD3-DKkiVeA")

🚀 LlamaIndex + Gemini Video Processing Ready!
✅ Multimodal RAG with video frames + transcript
✅ Using local embeddings (no OpenAI key needed)
✅ Professional document management
✅ Built-in error handling and retries
✅ Sophisticated retrieval and ranking

Usage:
result = demo_video_analysis('YOUR_YOUTUBE_URL')

💡 Perfect for AI startup demo!

🧪 Testing local embeddings...
✅ Local embeddings working! Vector dimension: 384
🧹 Deleted collection: video_analysis
✅ ChromaDB cleared!
🧹 Cleared work directory
✅ Session restarted - ready for fresh processing!

🎬 Processing video with LlamaIndex + Gemini: https://www.youtube.com/watch?v=kD3-DKkiVeA
1️⃣ Getting transcript...
✅ Retrieved 74 transcript segments
2️⃣ Downloading video and extracting frames...
🔄 Downloading video for frame extraction...
[youtube] Extracting URL: https://www.youtube.com/watch?v=kD3-DKkiVeA
[youtube] kD3-DKkiVeA: Downloading webpage
[youtube] kD3-DKkiVeA: Downloading tv client config
[youtube] kD3-DKkiVeA: Downloading tv 

In [ ]:
# Testing Your Video RAG System - Interactive Demo
# Run this after your video has been processed

# ============================================================================
# BASIC TESTING FUNCTIONS
# ============================================================================

def test_video_search(query_engine, test_queries=None):
    """Test the video search with various types of queries"""

    if test_queries is None:
        test_queries = [
            # Content questions
            "What is the main topic of this video?",
            "Summarize the key points discussed",
            "What are the most important takeaways?",

            # Specific search
            "When does the speaker mention specific tools or technologies?",
            "What examples are given in the video?",
            "Are there any numbers or statistics mentioned?",

            # Time-based queries
            "What happens in the first minute?",
            "What is discussed at the end of the video?",
            "What is the most important part?",

            # Follow-up questions
            "Can you explain that in simpler terms?",
            "What did you mean by that?",
            "Give me more details about the main concept"
        ]

    print("🔍 Testing Video Search & Chat")
    print("=" * 50)

    for i, query in enumerate(test_queries, 1):
        print(f"\n🤖 Query {i}: {query}")
        print("-" * 40)

        try:
            response = query_engine.query(query)
            print(f"📝 Response: {response}")

            # Show sources if available
            if hasattr(response, 'source_nodes') and response.source_nodes:
                print(f"\n📚 Sources ({len(response.source_nodes)} found):")
                for j, node in enumerate(response.source_nodes[:2]):  # Show top 2 sources
                    metadata = node.metadata
                    timestamp = metadata.get('timestamp_formatted', metadata.get('timestamp', 'unknown'))
                    print(f"   {j+1}. Timestamp: {timestamp}")
                    print(f"      Content: {node.text[:100]}...")

        except Exception as e:
            print(f"❌ Error: {e}")

        print("\n" + "="*50)

def interactive_chat(query_engine):
    """Interactive chat session with your video"""
    print("\n💬 Interactive Video Chat")
    print("=" * 30)
    print("Ask questions about your video! Type 'quit' to exit.")
    print("Example questions:")
    print("- What is this video about?")
    print("- When does X happen?")
    print("- Explain the main concept")
    print("- What are the key takeaways?")
    print()

    while True:
        query = input("🤔 Your question: ").strip()

        if query.lower() in ['quit', 'exit', 'bye']:
            print("👋 Chat ended!")
            break

        if not query:
            continue

        try:
            print("🤖 Thinking...")
            response = query_engine.query(query)
            print(f"📝 Answer: {response}")

            # Show timestamp if available
            if hasattr(response, 'source_nodes') and response.source_nodes:
                timestamps = []
                for node in response.source_nodes[:3]:
                    ts = node.metadata.get('timestamp_formatted', node.metadata.get('timestamp'))
                    if ts:
                        timestamps.append(str(ts))

                if timestamps:
                    print(f"🕐 Relevant timestamps: {', '.join(timestamps)}")

            print()

        except Exception as e:
            print(f"❌ Error: {e}")

# ============================================================================
# DEMO FUNCTIONS FOR DIFFERENT SCENARIOS
# ============================================================================

def demo_search_capabilities(query_engine):
    """Demonstrate different types of search capabilities"""

    demo_scenarios = {
        "📋 Content Summary": [
            "Give me a brief summary of this video",
            "What are the main points covered?",
            "What is the primary topic discussed?"
        ],

        "🔍 Specific Information": [
            "What specific examples are mentioned?",
            "Are there any tools or technologies discussed?",
            "What data or statistics are provided?"
        ],

        "⏰ Time-based Queries": [
            "What happens in the beginning?",
            "What is discussed towards the end?",
            "What is the most important moment?"
        ],

        "🎯 Deep Understanding": [
            "Explain the main concept in detail",
            "What problem is being solved?",
            "What are the implications of this topic?"
        ]
    }

    print("🎭 Video RAG Capability Demo")
    print("=" * 40)

    for category, queries in demo_scenarios.items():
        print(f"\n{category}")
        print("-" * 30)

        # Test one query from each category
        query = queries[0]
        print(f"🤖 Testing: '{query}'")

        try:
            response = query_engine.query(query)
            print(f"📝 Result: {response}")

            if hasattr(response, 'source_nodes') and response.source_nodes:
                print(f"📚 Found {len(response.source_nodes)} relevant sources")

        except Exception as e:
            print(f"❌ Error: {e}")

        print()

def benchmark_search_quality(query_engine):
    """Quick benchmark of search quality and response time"""
    import time

    benchmark_queries = [
        "What is this video about?",
        "Summarize the main points",
        "What happens at the beginning?",
        "What are the key takeaways?",
        "Explain the main concept"
    ]

    print("📊 Search Quality Benchmark")
    print("=" * 35)

    total_time = 0
    successful_queries = 0

    for i, query in enumerate(benchmark_queries, 1):
        print(f"\n{i}. Testing: '{query}'")

        start_time = time.time()
        try:
            response = query_engine.query(query)
            end_time = time.time()

            response_time = end_time - start_time
            total_time += response_time
            successful_queries += 1

            # Quick quality check
            response_length = len(str(response))
            has_sources = hasattr(response, 'source_nodes') and response.source_nodes

            print(f"   ✅ Response time: {response_time:.2f}s")
            print(f"   📝 Response length: {response_length} chars")
            print(f"   📚 Has sources: {has_sources}")

            if response_length < 50:
                print("   ⚠️  Short response - might need better query")

        except Exception as e:
            print(f"   ❌ Failed: {e}")

    if successful_queries > 0:
        avg_time = total_time / successful_queries
        print(f"\n📈 Benchmark Results:")
        print(f"   Success rate: {successful_queries}/{len(benchmark_queries)}")
        print(f"   Average response time: {avg_time:.2f}s")
        print(f"   System status: {'✅ Ready for demo!' if successful_queries >= 4 else '⚠️ Needs tuning'}")

# ============================================================================
# COMPLETE DEMO WORKFLOW
# ============================================================================

def full_demo(result):
    """Complete demo workflow after video processing"""

    query_engine = result.get("query_engine")
    if not query_engine:
        print("❌ No query engine found in result")
        return

    print("🎬 Complete Video RAG Demo")
    print("=" * 30)

    # Show video info
    if "metadata" in result:
        video_info = result["metadata"].get("video_info", {})
        print(f"📹 Video: {video_info.get('title', 'Unknown')}")
        print(f"⏱️  Duration: {video_info.get('duration', 0)} seconds")
        print()

    # 1. Basic capability test
    print("1️⃣ Testing basic search capabilities...")
    test_video_search(query_engine, [
        "What is this video about?",
        "Summarize the key points",
        "What happens in the first part?"
    ])

    # 2. Performance benchmark
    print("\n2️⃣ Running performance benchmark...")
    benchmark_search_quality(query_engine)

    # 3. Interactive mode
    print("\n3️⃣ Ready for interactive testing!")
    interactive_choice = input("Start interactive chat? (y/n): ").strip().lower()

    if interactive_choice == 'y':
        interactive_chat(query_engine)
    else:
        print("💡 You can run interactive_chat(result['query_engine']) anytime!")

    print("\n🎉 Demo complete! Your video RAG system is working!")

# ============================================================================
# QUICK START COMMANDS
# ============================================================================

def quick_test(result):
    """Quick test to verify everything works"""
    query_engine = result.get("query_engine")

    if not query_engine:
        print("❌ No query engine found")
        return False

    print("🚀 Quick Test")
    print("-" * 15)

    try:
        response = query_engine.query("What is this video about?")
        print(f"✅ System working! Response: {response}")
        return True
    except Exception as e:
        print(f"❌ System error: {e}")
        return False

# ============================================================================
# USAGE INSTRUCTIONS
# ============================================================================

print("🎯 How to Test Your Video RAG System")
print("=" * 40)
print()
print("After processing your video, you have several options:")
print()
print("1. 🚀 Quick test:")
print("   quick_test(result)")
print()
print("2. 💬 Interactive chat:")
print("   interactive_chat(result['query_engine'])")
print()
print("3. 🔍 Search capabilities demo:")
print("   demo_search_capabilities(result['query_engine'])")
print()
print("4. 📊 Full benchmark:")
print("   benchmark_search_quality(result['query_engine'])")
print()
print("5. 🎭 Complete demo:")
print("   full_demo(result)")
print()
print("💡 Start with quick_test() to make sure everything works!")

# Example workflow:
# 1. result = simple_transcript_rag("YOUR_VIDEO_URL")  # Process video
# quick_test(result)                                # Verify it works
# interactive_chat(result['query_engine'])          # Try interactive chat
full_demo(result)                                 # Complete demo

🎯 How to Test Your Video RAG System

After processing your video, you have several options:

1. 🚀 Quick test:
   quick_test(result)

2. 💬 Interactive chat:
   interactive_chat(result['query_engine'])

3. 🔍 Search capabilities demo:
   demo_search_capabilities(result['query_engine'])

4. 📊 Full benchmark:
   benchmark_search_quality(result['query_engine'])

5. 🎭 Complete demo:
   full_demo(result)

💡 Start with quick_test() to make sure everything works!
🎬 Complete Video RAG Demo
📹 Video: Why you feel stuck — and how to get motivated - Shannon Odell
⏱️  Duration: 300 seconds

1️⃣ Testing basic search capabilities...
🔍 Testing Video Search & Chat

🤖 Query 1: What is this video about?
----------------------------------------
📝 Response: The video discusses feeling stuck and how to find motivation, particularly when working towards large-scale and complex societal goals.  It highlights the importance of celebrating smaller wins along the way to combat feelings of powerlessness.


📚 Sources